In [1]:
#!/usr/bin/env python
# coding: utf-8

# Libraries and Filesystem Setup

import pandas as pd
import pickle
from sklearn.neighbors import NearestNeighbors
import numpy as np
import matplotlib.pyplot as plt
import scipy.sparse
import statistics
from random import sample
from sklearn.preprocessing import normalize
from scipy import spatial
import math
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
import pickle
kmeans_lda_10_false_full_5_cluster_list = pickle.load(open("appdetex_kmeans_lda_10_cluster_5-cluster_list.results", "rb"))


In [3]:
kmeans_lda_10_false_full_5_cluster_list

{0: [(0, 3),
  (0, 5),
  (0, 6),
  (0, 13),
  (0, 17),
  (0, 18),
  (0, 20),
  (0, 21),
  (0, 24),
  (0, 32),
  (0, 38),
  (0, 58),
  (0, 66),
  (0, 77),
  (0, 86),
  (0, 89),
  (0, 99),
  (0, 102),
  (0, 104),
  (0, 114),
  (0, 117),
  (0, 135),
  (0, 136),
  (0, 150),
  (0, 151),
  (0, 155),
  (0, 156),
  (0, 158),
  (0, 162),
  (0, 163),
  (0, 165),
  (0, 166),
  (0, 171),
  (0, 186),
  (0, 187),
  (0, 208),
  (0, 211),
  (0, 216),
  (0, 226),
  (0, 237),
  (0, 255),
  (0, 276),
  (0, 278),
  (0, 286),
  (0, 304),
  (0, 310),
  (0, 319),
  (0, 325),
  (0, 334),
  (0, 347),
  (0, 351),
  (0, 368),
  (0, 376),
  (0, 378),
  (0, 386),
  (0, 409),
  (0, 447),
  (0, 463),
  (0, 480),
  (0, 487),
  (0, 492),
  (0, 511),
  (0, 513),
  (0, 516),
  (0, 525),
  (0, 558),
  (0, 559),
  (0, 561),
  (0, 563),
  (0, 575),
  (0, 576),
  (0, 580),
  (0, 589),
  (0, 605),
  (0, 610),
  (0, 620),
  (0, 625),
  (0, 633),
  (0, 634),
  (0, 635),
  (0, 647),
  (0, 648),
  (0, 652),
  (0, 657),
  (0, 658

In [3]:
cluster0 = [t[1] for t in kmeans_lda_10_false_full_5_cluster_list[0]]
cluster1 = [t[1] for t in kmeans_lda_10_false_full_5_cluster_list[1]]
cluster2 = [t[1] for t in kmeans_lda_10_false_full_5_cluster_list[2]]
cluster3 = [t[1] for t in kmeans_lda_10_false_full_5_cluster_list[3]]
cluster4 = [t[1] for t in kmeans_lda_10_false_full_5_cluster_list[4]]

In [7]:
#cluster1

In [4]:
class knn_clustering_and_metrics:
    matrix_name = None
    df_merged = None
    matrix = None
    index_centroid = None
    
    distances = None
    indices = None
    
    distances0 = None
    indices0 = None
    
    distances1 = None
    indices1 = None
    
    distances2 = None
    indices2 = None
    
    distances3 = None
    indices3 = None
    
    distances4 = None
    indices4 = None
    
    algorithm = None
    num_neighbors = None
    similarities_sparse = None

    ###############
    intra_avg_list = []
    intra_avg_map = {}
    intra_variance_list = []
    intra_variance_map = {}
    
    ###############
    total_checks = 0
    total_checks_map = {}
    total_hits = 0
    total_hits_map = {}
    total_hit_percent = 0
    total_hit_percent_map = {}   
    total_position_sum = 0
    total_position_sum_map = {}
    total_avg_hit_pos = 0
    total_avg_hit_pos_map = {}
    
    ################
    paraphrase_checks = 0
    paraphrase_checks_map = {}
    paraphrase_hits = 0
    paraphrase_hits_map = {}
    paraphrase_hit_percent = 0
    paraphrase_hit_percent_map = {}   
    paraphrase_position_sum = 0
    paraphrase_position_sum_map = {}
    paraphrase_avg_hit_pos = 0
    paraphrase_avg_hit_pos_map = {}
    
    ################
    plagiarize_checks = 0
    plagiarize_checks_map = {}
    plagiarize_hits = 0
    plagiarize_hits_map = {}
    plagiarize_hit_percent = 0
    plagiarize_hit_percent_map = {}
    plagiarize_position_sum = 0
    plagiarize_position_sum_map = {}
    plagiarize_avg_hit_pos = 0    
    plagiarize_avg_hit_pos_map = {}
        
    def __init__(self, stemmed, representation, algorithm, num_neighbors):
        self.stemmed = stemmed
        self.representation = representation
        self.matrix_name = representation + '/'
        if stemmed == 'true':
            self.matrix_name += 'stemmed_' 
        self.matrix_name += representation + '_matrix'
        
        if algorithm not in ['ball_tree', 'kd_tree', 'brute', 'auto']:
            raise (algorithm + ' is not a valid option for algorithm argument')
        self.algorithm = algorithm
        self.num_neighbors = num_neighbors  
            
    def load_data(self):
        
        # file extension
        
        file_ext = ''
        if self.matrix_name in ['tfidf/tfidf_matrix', 'tfidf/stemmed_tfidf_matrix', 'bow/bow_matrix', 'bow/stemmed_bow_matrix']:
            file_ext = '.npz'
        else:
            file_ext = '.npy'
        print('using file extension ' + file_ext + ' for ' + self.matrix_name)
        
        # loading file
        
        matrix_file = "../../processed_files/" + self.matrix_name + file_ext
        print('loading ' + matrix_file)
        matrix = None
        if self.matrix_name in ['tfidf/tfidf_matrix', 'tfidf/stemmed_tfidf_matrix']:
            matrix = scipy.sparse.load_npz(matrix_file)
            sum_row = matrix.sum(axis=1)
            norm_matrix = matrix / sum_row.reshape(len(sum_row),1)
            self.matrix = norm_matrix
        #elif self.matrix_name != 'bow_matrix':
        elif self.matrix_name in ['bow/bow_matrix', 'bow/stemmed_bow_matrix']:
            self.matrix = scipy.sparse.load_npz(matrix_file)
        else:
            matrix = np.load(matrix_file, allow_pickle = True)
            sum_row = matrix.sum(axis=1)+1.0e-6
            norm_matrix = matrix / sum_row.reshape(len(sum_row),1)
            self.matrix = norm_matrix
        #else:
        #    self.matrix = np.load(matrix_file, allow_pickle = True)
            
        # loading raw data
        raw_data_file = '../../processed_files/df_merged.pickle'
        print('loading ' + raw_data_file)
        self.df_merged = pickle.load(open(raw_data_file, "rb"))

    def run_knn(self):
        print('finding nearest ' + str(self.num_neighbors) + ' neighbors with ' + self.algorithm + ' algorithm')
        # setting an array element with a sequence
        # should I call to list on the bow data before saving it?
        print('type of metrix: ' + str(type(self.matrix)))
        
        print('cluster0')
        nbrs0 = NearestNeighbors(self.num_neighbors, self.algorithm).fit(self.matrix[cluster0])
        distances0, indices0 = nbrs0.kneighbors(self.matrix[cluster0])

        print('cluster1')
        nbrs1 = NearestNeighbors(self.num_neighbors, self.algorithm).fit(self.matrix[cluster1])
        distances1, indices1 = nbrs1.kneighbors(self.matrix[cluster1])
        
        print('cluster2')
        nbrs2 = NearestNeighbors(self.num_neighbors, self.algorithm).fit(self.matrix[cluster2])
        distances2, indices2 = nbrs2.kneighbors(self.matrix[cluster2])

        print('cluster3')
        nbrs3 = NearestNeighbors(self.num_neighbors, self.algorithm).fit(self.matrix[cluster3])
        distances3, indices3 = nbrs3.kneighbors(self.matrix[cluster3])

        print('cluster4')
        nbrs4 = NearestNeighbors(self.num_neighbors, self.algorithm).fit(self.matrix[cluster4])
        distances4, indices4 = nbrs4.kneighbors(self.matrix[cluster4])
        
        self.distances = np.concatenate((distances0, distances1, distances2, distances3, distances4), axis=0)
        self.indices = np.concatenate((indices0, indices1, indices2, indices3, indices4), axis=0)
        
    def generate_raw_content_cluster_df(self, index):
        index_list = list(self, self.indices[index])
        distance_list = list(self.distances[index])
        cluster_seed = df_merged.loc[index].to_frame().T
        cluster_seed['DISTANCE'] = 0
        cluster_df = df_merged.loc[index_list[1:]]
        cluster_df['DISTANCE'] = distance_list[1:]
        combined = pd.concat([cluster_seed, cluster_df.sort_values(by='DISTANCE', ascending=True)])
        return combined.style.set_properties(subset=['text'], **{'width-min': '100px'})

    def calculate_intra_average_and_variance(self):
        print('calculating metrics for intracluster average and variance')
        self.intra_avg_list = np.mean(self.distances, axis=1)
        self.intra_variance_list = np.var(self.distances, axis=1)
        for i in range(len(self.distances)):
            self.intra_avg_map[i] = np.average(self.distances[i])
            self.intra_variance_map[i] = np.var(self.distances[i])
        
    def calculate_centroid(self, index):
        rows = self.matrix[index,:]
        row_mean = np.array(rows.mean(axis=0))
        if isinstance(row_mean[0], np.ndarray):
            return row_mean[0]
        else:
            return row_mean

    def generate_index_centroid_map(self):
        dist_mat = []
        for k in self.indices: 
            temp = self.calculate_centroid(k)
            dist_mat.append(temp)
        self.index_centroid = np.array(dist_mat)

    def sample_mean_intercluster_dist(self, sample_size):
        indices_sample = sample(range(self.matrix.shape[0]),sample_size)
        centroid_centroid_distance = {}
        for i in indices_sample:
            for j in indices_sample:
                if i < j:
                    key = str(i) + "::" + str(j)
                    cos_sim = None
                    cos_sim = 1 - sklearn.metrics.pairwise.cosine_similarity([self.index_centroid[i]], [self.index_centroid[j]])
                    centroid_centroid_distance[key] = cos_sim
        val = np.array(list(centroid_centroid_distance.values())).mean()
        if math.isnan(val):
            print('detected nan val')
            print(np.array(list(centroid_centroid_distance.values())))
        return val
        
    def calculate_total_hits_and_position(self):
        self.total_checks = 0
        self.total_hits = 0
        self.total_position_sum = 0
        for cluster in self.indices:
            if cluster[0] % 2 == 0:
                self.total_checks += 1
                self.total_checks_map[cluster[0]] = 1
                if (cluster[0]+1) in cluster:
                    self.total_hits += 1
                    self.total_hits_map[cluster[0]] = 1
                    
                    pos = list(cluster).index(cluster[0]+1)
                    self.total_position_sum += pos
                
                    self.total_hit_percent_map[cluster[0]] = 1
                    self.total_avg_hit_pos_map[cluster[0]] = pos
                
                self.total_hit_percent_map[cluster[0]] = 0
                self.total_avg_hit_pos_map[cluster[0]] = 0
        
        self.total_hit_percent = self.total_hits / self.total_checks
        self.total_avg_hit_pos = self.total_position_sum / self.total_hits
        
    def calculate_paraphrase_hits_and_position(self):
        self.paraphrase_checks = 0
        self.paraphrase_hits = 0
        self.paraphrase_position_sum = 0
        paraphrase_indices = list(self.df_merged['paraphrase'][self.df_merged['paraphrase'] == True].index)
        for cluster in self.indices:
            if cluster[0] % 2 == 0 and cluster[0] in paraphrase_indices:
                self.paraphrase_checks += 1
                self.paraphrase_checks_map[cluster[0]] = 1
                if (cluster[0]+1) in cluster:
                    self.paraphrase_hits += 1
                    self.total_hits_map[cluster[0]] = 1
                    
                    pos = list(cluster).index(cluster[0]+1)
                    self.paraphrase_position_sum += pos
                    
                    self.paraphrase_hit_percent_map[cluster[0]] = 1
                    self.paraphrase_avg_hit_pos_map[cluster[0]] = pos
                
                self.paraphrase_hit_percent_map[cluster[0]] = 0
                self.paraphrase_avg_hit_pos_map[cluster[0]] = 0
                
        self.paraphrase_hit_percent = self.paraphrase_hits / self.paraphrase_checks
        self.paraphrase_avg_hit_pos = self.paraphrase_position_sum / self.paraphrase_hits
        
    def calculate_plagiarize_hits_and_position(self):
        self.plagiarize_checks = 0
        self.plagiarize_hits = 0
        self.plagiarize_position_sum = 0
        plagiarize_indices = list(self.df_merged['paraphrase'][self.df_merged['paraphrase'] == False].index)
        for cluster in self.indices:
            if cluster[0] % 2 == 0 and cluster[0] in plagiarize_indices:
                self.plagiarize_checks += 1
                if (cluster[0]+1) in cluster:
                    self.plagiarize_hits += 1
                    self.plagiarize_hits_map[cluster[0]] = 1

                    pos = list(cluster).index(cluster[0]+1)
                    self.plagiarize_position_sum += pos
                    self.plagiarize_position_sum_map[cluster[0]] = pos
                    
                    self.plagiarize_hit_percent_map[cluster[0]] = 1
                    self.plagiarize_avg_hit_pos_map[cluster[0]] = pos
                    
                self.plagiarize_hit_percent_map[cluster[0]] = 0
                self.plagiarize_avg_hit_pos_map[cluster[0]] = 0
                    
        self.plagiarize_hit_percent = self.plagiarize_hits / self.plagiarize_checks
        self.plagiarize_avg_hit_pos = self.plagiarize_position_sum / self.plagiarize_hits
        
class bow_model(knn_clustering_and_metrics):
    def __init__(self, stemmed, algorithm, num_neighbors):
        super().__init__(stemmed, 'bow', algorithm, num_neighbors)


In [5]:
def calculate_total_hits_and_position(clusters):
    total_checks = 0
    total_hits = 0
    for cluster in clusters:
        if cluster[0] % 2 == 0:
            total_checks += 1
            if (cluster[0]+1) in cluster:
                total_hits += 1
    total_hit_percent = total_hits / total_checks

def calculate_paraphrase_hits_and_position(clusters):
    paraphrase_checks = 0
    paraphrase_hits = 0
    paraphrase_indices = list(knn_results.df_merged['paraphrase'][knn_results.df_merged['paraphrase'] == True].index)
    for cluster in clusters:
        if cluster[0] % 2 == 0 and cluster[0] in paraphrase_indices:
            paraphrase_checks += 1
            paraphrase_checks_map[cluster[0]] = 1
            if (cluster[0]+1) in cluster:
                paraphrase_hits += 1
    paraphrase_hit_percent = paraphrase_hits / paraphrase_checks

def calculate_plagiarize_hits_and_position(clusters):
    plagiarize_checks = 0
    plagiarize_hits = 0
    plagiarize_indices = list(knn_results.df_merged['paraphrase'][knn_results.df_merged['paraphrase'] == False].index)
    for cluster in clusters:
        if cluster[0] % 2 == 0 and cluster[0] in plagiarize_indices:
            plagiarize_checks += 1
            if (cluster[0]+1) in cluster:
                plagiarize_hits += 1
    plagiarize_hit_percent = plagiarize_hits / plagiarize_checks

In [6]:
from AppDetexKnnMetricResults import KnnMetricResults

def calculate_and_save_metrics(knn_results, time_1, time_2):
    # interdistance mean
    print('calculating interdistance mean')
    #knn_results_inter_dist_sample = [knn_results.sample_mean_intercluster_dist(500) for i in range(5)]
    #knn_results_inter_dist_mean = np.average(knn_results_inter_dist_sample)
    knn_results_inter_dist_mean = 0
    
    # silhouette coefficient
    #print('calculating silhouettte')
    #a = np.mean(knn_results.intra_avg_list)
    #b = knn_results_inter_dist_mean
    #silhouette_score = (b - a)/np.max([a, b])
    silhouette_score = -1
    
    # creating and saving result object
    print('creating metrics object')
    knnMetricResults = KnnMetricResults(stemmed, representation, technique, num_neighbors)
    knnMetricResults.set_intra_metrics(np.average(knn_results.intra_avg_list), knn_results.intra_avg_map, np.average(knn_results.intra_variance_list), knn_results.intra_variance_map)    
    #knnMetricResults.set_inter_metrics(knn_results_inter_dist_sample, knn_results_inter_dist_mean)
    #knnMetricResults.set_silhouette(silhouette_score)
    
    # calculate hits
    #knn_results.calculate_total_hits_and_position()
    #knn_results.calculate_paraphrase_hits_and_position()
    #knn_results.calculate_plagiarize_hits_and_position()

    # save hits
    #knnMetricResults.set_total_hit_metrics(knn_results.total_checks, knn_results.total_checks_map, knn_results.total_hits, knn_results.total_hits_map, knn_results.total_hit_percent, knn_results.total_hit_percent_map, knn_results.total_position_sum, knn_results.total_position_sum_map, knn_results.total_avg_hit_pos, knn_results.total_avg_hit_pos_map)
    #knnMetricResults.set_paraphrase_hit_metrics(knn_results.paraphrase_checks, knn_results.paraphrase_checks_map, knn_results.paraphrase_hits, knn_results.paraphrase_hits_map, knn_results.paraphrase_hit_percent, knn_results.paraphrase_hit_percent_map, knn_results.paraphrase_position_sum, knn_results.paraphrase_position_sum_map, knn_results.paraphrase_avg_hit_pos, knn_results.paraphrase_avg_hit_pos_map)
    #knnMetricResults.set_plagiarize_hit_metrics(knn_results.plagiarize_checks, knn_results.plagiarize_checks_map, knn_results.plagiarize_hits, knn_results.plagiarize_hits_map, knn_results.plagiarize_hit_percent, knn_results.plagiarize_hit_percent_map, knn_results.plagiarize_position_sum, knn_results.paraphrase_position_sum_map, knn_results.paraphrase_avg_hit_pos, knn_results.plagiarize_avg_hit_pos_map)

    # set computing time metrics
    #time_3 = round(time.time() * 1000)
    #knnMetricResults.set_compute_time(time_2-time_1, time_3-time_2)
    
    print('saving metrics object')
    knnMetricResults.save()

In [7]:
def new_bow_knn():
    bow_knn = bow_model(stemmed, technique, num_neighbors)
    bow_knn.load_data()
    time_1 = round(time.time() * 1000)
    bow_knn.run_knn()
    time_2 = round(time.time() * 1000)
    bow_knn.calculate_intra_average_and_variance()
    bow_knn.generate_index_centroid_map()
    return bow_knn, time_1, time_2

In [8]:
stemmed = 'true'
representation = 'BOW'
technique = 'kd_tree'
num_neighbors = 5

In [9]:
import time
knn_results, time_1, time_2 = new_bow_knn()

using file extension .npz for bow/stemmed_bow_matrix
loading ../../processed_files/bow/stemmed_bow_matrix.npz
loading ../../processed_files/df_merged.pickle
finding nearest 5 neighbors with kd_tree algorithm
type of metrix: <class 'scipy.sparse.csr.csr_matrix'>
cluster0
cluster1


/Users/dkarsann/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass n_neighbors=5, radius=kd_tree as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


cluster2
cluster3
cluster4
calculating metrics for intracluster average and variance


In [10]:
calculate_and_save_metrics(knn_results, time_1, time_2)

calculating interdistance mean
creating metrics object
saving metrics object


In [11]:
len(knn_results.intra_avg_map)

12968

In [12]:
knn_results.intra_avg_map[0]

4.732863826479693

In [13]:
len(knn_results.intra_avg_map)

12968

In [14]:
knn_results.indices[0]

array([   0,  583, 1169,  226, 1373])

In [15]:
knn_results.indices[1]

array([ 1, 21, 13, 23, 14])

In [16]:
knn_results.indices[2]

array([28,  6,  5,  7,  2])

In [17]:
knn_results.indices[3]

array([ 1, 21, 13, 23, 14])

In [18]:
knn_results.indices[4]

array([ 1, 21, 13, 23, 14])

## building intra dist -- cluster list

In [19]:
intraDistClusterList = []

for i in range(len(knn_results.intra_avg_map)):
    intraDistClusterList.append((knn_results.intra_avg_map[i], list(knn_results.indices[i])))
    
intraDistClusterList = sorted(intraDistClusterList)

In [20]:
intraDistClusterList

[(0.0, [1, 21, 13, 23, 14]),
 (0.0, [1, 21, 13, 23, 14]),
 (0.0, [1, 21, 13, 23, 14]),
 (0.0, [1, 21, 13, 23, 14]),
 (0.0, [1, 21, 13, 23, 14]),
 (0.0, [1, 21, 13, 23, 14]),
 (0.0, [1, 21, 13, 23, 14]),
 (0.0, [1, 21, 13, 23, 14]),
 (0.0, [28, 6, 5, 7, 2]),
 (0.0, [28, 6, 5, 7, 2]),
 (0.0, [28, 6, 5, 7, 2]),
 (0.0, [28, 6, 5, 7, 2]),
 (0.0, [28, 6, 5, 7, 2]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76]),
 (0.0, [141, 398, 53, 29, 76])

In [21]:
pickle.dump(intraDistClusterList, open('appdetex_knn_on_kmeans-intra_dist_cluster_list.results', 'wb'))
